In [1]:
import polars as pl
import duckdb


In [2]:
pl_df = pl.read_csv('data/portal_upload/lnrs-all-grants-tbl.csv', separator=";")

In [3]:
pl_df.glimpse()

Rows: 283
Columns: 6
$ url            <str> https://www.gov.uk/countryside-stewardship-grants/administration-of-group-managed-agreements-supplement-sp10, https://www.gov.uk/countryside-stewardship-grants/livestock-exclusion-supplement-scrub-and-successional-areas-wd9, https://www.gov.uk/countryside-stewardship-grants/creation-of-successional-areas-and-scrub-wd8, https://www.gov.uk/countryside-stewardship-grants/management-of-successional-areas-and-scrub-wd7, https://www.gov.uk/countryside-stewardship-grants/protection-of-in-field-trees-on-arable-land-be1, https://www.gov.uk/countryside-stewardship-grants/supplement-for-restorative-pruning-of-fruit-trees-be7, https://www.gov.uk/countryside-stewardship-grants/maintenance-of-weatherproof-traditional-farm-buildings-hs1, https://www.gov.uk/countryside-stewardship-grants/take-historic-and-archaeological-features-out-of-cultivation-hs2, https://www.gov.uk/countryside-stewardship-grants/management-of-historic-and-archaeological-features-on-gra

In [37]:
con = duckdb.connect('data/lnrs_db.duckdb')

Grants


In [52]:
con.sql(
    '''
CREATE TABLE lnrs_all_grants_tbl (url VARCHAR, grant_name VARCHAR NOT NULL UNIQUE, grant_id VARCHAR, grant_scheme VARCHAR, annual_payment VARCHAR, grant_focus VARCHAR);

    '''
)

Funding Schemes

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_area_funding_schemes_tbl (area_id INTEGER,
funding_schemes VARCHAR,
FOREIGN KEY area_id REFERENCES lnrs_areas_tbl (area_id));
    '''
)

Areas

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_areas_tbl (
    area_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    area_name VARCHAR,
    area_description VARCHAR,
    area_link VARCHAR
);
    '''
)

Measures Priority Areas Lookup

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_measures_priority_area_lookup_tbl(
    measure_id INTEGER,
    priority_id INTEGER,
    area_id INTEGER,
    link_or_code VARCHAR,
    FOREIGN KEY measure_id REFERENCES lnrs_measures_tbl (measure_id),
    FOREIGN KEY priority_id REFERENCES lnrs_priorities_tbl (priority_id),
    FOREIGN KEY area_id REFERENCES lnrs_areas_tbl (area_id)
);
    '''
)

Measures

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_measures_tbl(
    measure_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    measure VARCHAR,
    ambition VARCHAR,
    land_type VARCHAR
);
    '''
)

Priorities

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_priorities_tbl(
    priority_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    theme VARCHAR,
    biodiversity_priority VARCHAR
);
    '''
)

Priority Area Lookup

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_priority_area_lookup_tbl(
    priority_id INTEGER,
    area_id INTEGER,
    FOREIGN KEY priority_id REFERENCES lnrs_priorities_tbl (priority_id),
    FOREIGN KEY area_id REFERENCES lnrs_areas_tbl (area_id)
);
    '''
)

Species Area Priority Lookup

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_species_area_priority_lookup_tbl(
    species_id INTEGER,
    priority_id INTEGER,
    area_id INTEGER,
    FOREIGN KEY species_id REFERENCES species_tbl (species_id),
    FOREIGN KEY priority_id REFERENCES lnrs_priorities_tbl (priority_id),
    FOREIGN KEY area_id REFERENCES lnrs_areas_tbl (area_id)
);
    '''
)

Species

In [ ]:
con.sql(
    '''
CREATE TABLE lnrs_species_tbl(
 species_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
 taxa VARCHAR,
 assemblage VARCHAR,
 usageKey INTEGER,
 scientificName VARCHAR,
 canonicalName VARCHAR,
 status VARCHAR,
 kingdom VARCHAR,
 phylum VARCHAR,
 order VARCHAR,
 family VARCHAR,
 genus VARCHAR,
 species VARCHAR,
 kingdomKey INTEGER,
 phylumKey INTEGER,
 classKey INTEGER,
 orderKey INTEGER,
 familyKey INTEGER,
 genusKey INTEGER,
 speciesKey INTEGER,
 synonym BOOL,
 class VARCHAR,
 acceptedUsageKey INTEGER,
 verbatim_name VARCHAR,
 gbif_species_url VARCHAR
    
);
    '''
)

In [64]:
cols = pl.read_csv('data/portal_upload/lnrs-species-tbl.csv', separator=';').columns
dtypes = pl.read_csv('data/portal_upload/lnrs-species-tbl.csv', separator=';').dtypes

In [70]:
dict(zip(cols, dtypes))

{'species_id': Int64,
 'taxa': Utf8,
 'assemblage': Utf8,
 'usageKey': Int64,
 'scientificName': Utf8,
 'canonicalName': Utf8,
 'status': Utf8,
 'kingdom': Utf8,
 'phylum': Utf8,
 'order': Utf8,
 'family': Utf8,
 'genus': Utf8,
 'species': Utf8,
 'kingdomKey': Int64,
 'phylumKey': Int64,
 'classKey': Int64,
 'orderKey': Int64,
 'familyKey': Int64,
 'genusKey': Int64,
 'speciesKey': Int64,
 'synonym': Boolean,
 'class': Utf8,
 'acceptedUsageKey': Int64,
 'verbatim_name': Utf8,
 'gbif_species_url': Utf8}

In [53]:
con.sql(
'''
COPY lnrs_all_grants_tbl FROM 'data/portal_upload/lnrs-all-grants-tbl.csv'(DELIMITER ';', HEADER)

'''


)

In [54]:
con.sql('SELECT * FROM lnrs_all_grants_tbl')

┌──────────────────────┬──────────────────────┬──────────┬───┬──────────────────┬──────────────────────┐
│         url          │      grant_name      │ grant_id │ … │  annual_payment  │     grant_focus      │
│       varchar        │       varchar        │ varchar  │   │     varchar      │       varchar        │
├──────────────────────┼──────────────────────┼──────────┼───┼──────────────────┼──────────────────────┤
│ https://www.gov.uk…  │ SP10: Administrati…  │ SP10     │ … │ NULL             │ Additional supplem…  │
│ https://www.gov.uk…  │ WD9: Livestock exc…  │ WD9      │ … │ NULL             │ Woodland and scrub   │
│ https://www.gov.uk…  │ WD8: Creation of s…  │ WD8      │ … │ NULL             │ Woodland and scrub   │
│ https://www.gov.uk…  │ WD7: Management of…  │ WD7      │ … │ NULL             │ Woodland and scrub   │
│ https://www.gov.uk…  │ BE1: Protection of…  │ BE1      │ … │ NULL             │ Boundaries, trees …  │
│ https://www.gov.uk…  │ BE7: Supplement fo…  │ BE7    

In [45]:
con.sql('''
        CREATE TABLE lnrs_all_grants_tbl AS 
        SELECT * FROM 
        read_csv(
        'data/portal_upload/lnrs-all-grants-tbl.csv',
        header = true,
        delim = ';',
        columns = {'url': 'VARCHAR',
                'grant_name': 'VARCHAR',
                'grant_id': 'VARCHAR',
                'grant_scheme': 'VARCHAR',
                'annual_payment': 'VARCHAR',
                'grant_focus': 'VARCHAR'}
        ) 
        
        ''')

ParserException: Parser Error: syntax error at or near "("
LINE 15:         (url VARCHAR, grant_name VARCHAR, grant_id VARCHAR, grant_scheme VARCHAR, annual_payment VARCHAR, grant_focus VARCHAR)
        ...
                 ^

In [51]:
con.sql('''
       DROP TABLE lnrs_all_grants_tbl 
        
        
'''
        )

In [41]:
con.sql('SELECT * FROM lnrs_all_grants_tbl')

┌──────────────────────┬──────────────────────┬──────────┬───┬──────────────────┬──────────────────────┐
│         url          │      grant_name      │ grant_id │ … │  annual_payment  │     grant_focus      │
│       varchar        │       varchar        │ varchar  │   │     varchar      │       varchar        │
├──────────────────────┼──────────────────────┼──────────┼───┼──────────────────┼──────────────────────┤
│ https://www.gov.uk…  │ SP10: Administrati…  │ SP10     │ … │ NULL             │ Additional supplem…  │
│ https://www.gov.uk…  │ WD9: Livestock exc…  │ WD9      │ … │ NULL             │ Woodland and scrub   │
│ https://www.gov.uk…  │ WD8: Creation of s…  │ WD8      │ … │ NULL             │ Woodland and scrub   │
│ https://www.gov.uk…  │ WD7: Management of…  │ WD7      │ … │ NULL             │ Woodland and scrub   │
│ https://www.gov.uk…  │ BE1: Protection of…  │ BE1      │ … │ NULL             │ Boundaries, trees …  │
│ https://www.gov.uk…  │ BE7: Supplement fo…  │ BE7    

In [36]:
con.close()